In [16]:
# from google.colab import drive
# drive.mount('/content/drive')

# import sys
# import os
# py_file_location = "./drive/My Drive/LAB/COMP90086"
# sys.path.append(os.path.abspath(py_file_location))

# !pip install einops

In [1]:
from environment import *
from utils import *
from model import *

/Applications/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# ROOT = './drive/My Drive/LAB/COMP90086/COMP90086_2023_TLLdataset 2'
ROOT = './COMP90086_2023_TLLdataset 2'
seed = 19260817
data_list = pd.read_csv(ROOT + '/train.csv')
future_list = pd.read_csv(ROOT + '/test_candidates.csv')

In [3]:
imgs = dict()
for img in tqdm(os.listdir(ROOT+'/train/left')):
  imgs[img] = prepare_image(ROOT+'/train/left/'+img)

for img in tqdm(os.listdir(ROOT+'/train/right')):
  imgs[img] = prepare_image(ROOT+'/train/right/'+img)

for img in tqdm(os.listdir(ROOT+'/test/left')):
  imgs[img] = prepare_image(ROOT+'/test/left/'+img)

for img in tqdm(os.listdir(ROOT+'/test/right')):
  imgs[img] = prepare_image(ROOT+'/test/right/'+img)

  0%|          | 0/2000 [00:00<?, ?it/s]

100%|██████████| 2000/2000 [00:02<00:00, 896.19it/s]


In [4]:
train_list, valtest_list = train_test_split(data_list, test_size=0.3, random_state=seed) # train is 70%
val_list, test_list = train_test_split(valtest_list, test_size=0.5, random_state=seed) # val is 15%, test is 15%

In [6]:
class CNN_Key_Query_Categorisation_CFG:
    num_false               = 1 # how many false samples for each true sample
    target                  = 1 # target loss for correct. false will be 1-target
    encoder                 = 'resnet18' # the name of the model we want to load from torchhub
    freeze_encoder          = True
    input_shape             = (64, 3, 64, 64) # (batch size, 3, [what our loaded in images look like])
    num_mlp_layers          = 1 # how many mlp layers in the end. 0 means no mlp and just 1 out layer; n means (n+1)mlp + out
    hidden_dim              = 512 # hidden dim of mlp
    dropout                 = 0.1 # dropout rate of mlp
    res_learning            = False # whether to use residual layer in mlp
    pretrained              = True # whether to use pretrained encoder
    # ------------------------ #
    random_state            = seed
    lr                      = 1e-5
    loss                    = nn.BCELoss()   
    ROOT                    = ROOT # rootpath for loading images
    rootpath                = './' # rootpath for saving models
    # rootpath                = py_file_location
    DataFactory_Categorisation = DataFactory_Categorisation
    DataLoader_Categorisation = DataLoader_Categorisation
    images                  = imgs
    name                    = 'resnet18_categorisation'

cnn_key_query_categorisation = CNN_Key_Query_Categorisation(CNN_Key_Query_Categorisation_CFG)
cnn_key_query_categorisation.fit(train_list, val_list, batch_size = 64)

Using cache found in /Users/tg.chenny/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /Users/tg.chenny/.cache/torch/hub/pytorch_vision_v0.10.0
100%|██████████| 44/44 [08:04<00:00, 11.02s/it]


Epoch:   1 - Loss: 0.736, Accuracy: 0.495
Validation Loss: 0.725, Accuracy: 0.455


100%|██████████| 44/44 [07:55<00:00, 10.80s/it]


Epoch:   2 - Loss: 0.724, Accuracy: 0.497
Validation Loss: 0.722, Accuracy: 0.493


100%|██████████| 44/44 [08:56<00:00, 12.20s/it]


Epoch:   3 - Loss: 0.718, Accuracy: 0.500
Validation Loss: 0.721, Accuracy: 0.497


100%|██████████| 44/44 [08:02<00:00, 10.96s/it]


Epoch:   4 - Loss: 0.717, Accuracy: 0.499
Validation Loss: 0.713, Accuracy: 0.483


 14%|█▎        | 6/44 [01:25<09:03, 14.30s/it]


KeyboardInterrupt: 

In [ ]:
class CNN_Key_Query_Triplet_CFG:
    num_false               = 1 # how many hard negative to try for each anchor
    encoder                 = 'resnet18'
    freeze_encoder          = True
    input_shape             = (64, 3, 245, 200)
    num_mlp_layers          = 1
    hidden_dim              = 512
    dropout                 = 0.1
    res_learning            = False
    embed_dim               = 64
    pretrained              = True      
    # ------------------------ #
    random_state            = seed
    lr                      = 1e-5
    loss                    = nn.TripletMarginLoss() 
    ROOT                    = ROOT # rootpath for loading images
    rootpath                = './' # rootpath for saving
    # rootpath                = py_file_location
    DataFactory_Triplet     = DataFactory_Triplet
    DataLoader_Triplet      = DataLoader_Triplet
    images                  = imgs
    name                    = 'resnet18_triplet'

cnn_key_query_triplet = CNN_Key_Query_Triplet(CNN_Key_Query_Triplet_CFG)
cnn_key_query_triplet.fit(train_list, val_list, batch_size = 64)

# TODO: should we batch normalise...

In [11]:
def DataFactory_Triplet_future_right(train_list, num_false, seed, ranking = None):
    """ 
        Input:
            Num_False: number of false samples for each true sample
            Ranking: ranking of negative samples
            Target: target value for positive samples    
        Output:
            x_list: list of tuples (left, right)
    """
    
    right = list(train_list['right'])

    x_list = []

    np.random.seed(seed)

    for i in range(len(train_list)):
        
        right_tmp = [right[j] for j in range(len(right)) if j != i]
    
        sampled_right = np.random.choice(right_tmp, num_false, replace = False)

        x_list.append(right[i])

        for j in range(num_false):

            x_list.append((sampled_right[j]))

    return x_list

In [ ]:
def DataFactory_Triplet_future_left(train_list, num_false, seed, ranking = None):
    """ 
        Input:
            Num_False: number of false samples for each true sample
            Ranking: ranking of negative samples
            Target: target value for positive samples    
        Output:
            x_list: list of tuples (left, right)
    """
    
    left = list(train_list['left'])

    x_list = []

    for i in range(len(train_list)):
        
        x_list.append(left[i])

    return x_list

In [ ]:
def DataLoader_Triplet_future(x_list, batch_number, batch_size, CFG):
    
    x_ = []

    x_ = x_[batch_number*batch_size:(batch_number+1)*batch_size]

    for i in range(len(x_list)):
        x_.append(CFG.images[f'{x_list[i]}.jpg'])

    x_ = np.array(x_)

    return torch.FloatTensor(x_)